In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt, seaborn as sns
from kmodes.kprototypes import KPrototypes

In [ ]:
acc = pd.read_csv(os.path.join(dirname, filename))
acc.head()

In [ ]:
acc.info()

In [ ]:
null_cols = pd.DataFrame(100*acc.isnull().sum()/len(acc), columns=['nullPercent'])
null_cols

In [ ]:
df = acc.drop(list(null_cols[null_cols.nullPercent>50].index)+['Crash ID'], axis=1)
df.info()

In [ ]:
df.head()

In [ ]:
MonthDict={ 1 : "January",
       2 : "February",
       3 : "March",
       4 : "April",
       5 : "May",
       6 : "June",
       7 : "July",
       8 : "August",
       9 : "September",
       10 : "October",
       11 : "November",
       12 : "December"
}

df.Month = df.Month.map(MonthDict)
df.Year = df.Year.astype('str')
df['Speed Limit'] = pd.to_numeric(df['Speed Limit'], errors='coerce')
df = df.dropna()
df.info()

In [ ]:
obj_cols_index_no = [i for i,j in enumerate(df.columns) if j in df.select_dtypes(include='object').columns]
obj_cols_index_no

In [ ]:
no_of_clusters = np.arange(2,9)
costs = []
for i in no_of_clusters:
    model = KPrototypes(n_clusters=i, n_init=5, random_state=50, n_jobs=-1, verbose=1)
    model.fit_predict(np.array(df), categorical=obj_cols_index_no)
    costs.append(model.cost_)

In [ ]:
plt.plot(no_of_clusters, costs)
plt.show()

In [ ]:
model = KPrototypes(n_clusters=4, n_init=5, random_state=50, n_jobs=-1, verbose=1)
clusters = model.fit_predict(np.array(df), categorical=obj_cols_index_no)

In [ ]:
df['cluster_id'] = clusters
df.head()

In [ ]:
acc = acc.join(df.cluster_id)
acc = acc.iloc[acc.cluster_id.dropna().index]
acc.head()

In [ ]:
sns.boxplot(x=acc.cluster_id,y=acc['Age'])

In [ ]:
acc.groupby('cluster_id')['Crash Type'].value_counts().plot.bar()

In [ ]:
nan